In [ ]:
# Importa el módulo 'os' para acceder a variables de entorno del sistema
import os

# Importa 'load_dotenv' para cargar las variables de entorno desde un archivo .env
from dotenv import load_dotenv

# Importa la clase OpenAI del paquete openai
from openai import OpenAI

# Importa librerías estándar para hacer peticiones HTTP, manejar JSON y controlar el tiempo
import requests
import json
import time

# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Crea una instancia del cliente de OpenAI usando la clave API almacenada en la variable de entorno
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ID del asistente previamente creado en la plataforma de OpenAI
assistant_id = "asst_etc"

# Crea un nuevo hilo de conversación (thread) para interactuar con el asistente
thread = client.beta.threads.create()
print(f"Hilo creado con ID: {thread.id}")

# Envía un mensaje al hilo como si fuera el usuario, con una pregunta matemática
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",  # Especifica que el mensaje es del usuario
    content="¿Cuánto es 16284+991893-771939*12456? Puedes ejecutar código Python para esto"
)

# Inicia la ejecución del asistente con el mensaje dado
print("Ejecutando el asistente")
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)

# Espera a que el asistente termine de procesar la respuesta
while True:
    # Consulta el estado actual de la ejecución
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    # Si la ejecución ha terminado, se sale del bucle
    if run.status == "completed":
        print("Se completó la respuesta")
        break
    # Si no ha terminado, espera 1 segundo antes de volver a comprobar
    time.sleep(1)

# Si la ejecución se completó, obtiene los pasos realizados por el asistente
if run.status == "completed":
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id
    )

    # Recorre cada paso realizado por el asistente
    for step in run_steps:
        # Verifica si se usaron herramientas como el interprete de código
        if step.step_details.type == "tool_calls":
            for tool_call in step.step_details.tool_calls:
                # Si se utilizó el interprete de código, imprime el código que se ejecutó
                if tool_call.type == "code_interpreter":
                    print("Código Python")
                    print(tool_call.code_interpreter.input)
    
    # Obtiene el último mensaje del asistente en el hilo
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        order="desc",  # Orden descendente, para obtener el más reciente primero
        limit=1         # Solo un mensaje
    )
    
    # Muestra el contenido del mensaje si fue generado por el asistente
    for msg in messages:
        if msg.role == "assistant":
            for content_block in msg.content:
                print(content_block.text.value)  # Muestra la respuesta final del asistente


Hilo creado con ID: thread_5I6h3tZwjDR4O0izTOJrR2hD
Ejecutando el asistente
Se completó la respuesta
Código Python
# Calculamos la expresión matemática
resultado = 16284 + 991893 - 771939 * 12456
resultado
El resultado de la expresión \(16284 + 991893 - 771939 \times 12456\) es \(-9,614,264,007\).
